# Apartado 1 — Explicación del conjunto de datos (T1D Guertin 2024)

## Objetivo
- Identificar la temática del dataset y su interés analítico.
- Confirmar los dos archivos de trabajo: `survey_data_and_results_final.xlsx` y `assay_final_genotyping_file.xlsx`.
- Mostrar tamaño, estructura y relación entre tablas mediante una clave común.
- Declarar explícitamente que en genotipado el valor `-9` representa missing.

## Contexto mínimo (sin inventar resultados)
- Tema: cribado de riesgo de diabetes tipo 1 (T1D Screening Project, Guertin 2024).
- Tablas objetivo para análisis relacional: survey y genotyping.
- Clave prevista de JOIN: `SUBJECT_ID` (o equivalente tras ver columnas reales).

In [ ]:
import os
import shutil
import pandas as pd

# 1) Clonar repo automáticamente
!rm -rf BDA_dataset
!git clone https://github.com/kachytronico/BDA_dataset

# 2) Definir rutas reales del dataset dentro del repo clonado
base_path = "/content/BDA_dataset/Dataset not incorporated into the T1DKP"
survey_xlsx = os.path.join(base_path, "survey_data_and_results_final.xlsx")
genotyping_xlsx = os.path.join(base_path, "assay_final_genotyping_file.xlsx")

assert os.path.exists(survey_xlsx), f"Falta archivo requerido: {survey_xlsx}"
assert os.path.exists(genotyping_xlsx), f"Falta archivo requerido: {genotyping_xlsx}"

# 3) Copiar esos 2 XLSX a /content/data/raw/
raw_path = "/content/data/raw"
os.makedirs(raw_path, exist_ok=True)

survey_raw_xlsx = os.path.join(raw_path, "survey_data_and_results_final.xlsx")
genotyping_raw_xlsx = os.path.join(raw_path, "assay_final_genotyping_file.xlsx")

shutil.copy2(survey_xlsx, survey_raw_xlsx)
shutil.copy2(genotyping_xlsx, genotyping_raw_xlsx)

# 4) Convertir ambos a CSV en /content/data/csv/
csv_path = "/content/data/csv"
os.makedirs(csv_path, exist_ok=True)

survey_csv = os.path.join(csv_path, "survey.csv")
genotyping_csv = os.path.join(csv_path, "genotyping.csv")

df_survey = pd.read_excel(survey_raw_xlsx)
df_genotyping = pd.read_excel(genotyping_raw_xlsx)

df_survey.to_csv(survey_csv, index=False)
df_genotyping.to_csv(genotyping_csv, index=False)

# 5) Evidencia mínima obligatoria
!ls -lh BDA_dataset
!ls -lh /content/data/raw
!ls -lh /content/data/csv

print("shape survey:", df_survey.shape)
print("shape genotyping:", df_genotyping.shape)

print("\nhead survey:")
display(df_survey.head())

print("\nhead genotyping:")
display(df_genotyping.head())

# 6) Validación mínima del JOIN (sin inventar columnas comunes)
df_survey_str = pd.read_excel(survey_raw_xlsx, dtype=str)
df_genotyping_str = pd.read_excel(genotyping_raw_xlsx, dtype=str)

assert "SUBJECT_ID" in df_survey_str.columns, "No existe columna requerida SUBJECT_ID en survey."
assert "FID" in df_genotyping_str.columns, "No existe columna requerida FID en genotyping."

survey_ids = df_survey_str["SUBJECT_ID"].fillna("").str.strip()
geno_ids = df_genotyping_str["FID"].fillna("").str.strip()

survey_ids_non_empty = set(survey_ids[survey_ids != ""])
geno_ids_non_empty = set(geno_ids[geno_ids != ""])
ids_intersection = survey_ids_non_empty.intersection(geno_ids_non_empty)

merge_inner = (
    df_survey_str[["SUBJECT_ID"]]
    .assign(SUBJECT_ID=lambda d: d["SUBJECT_ID"].fillna("").str.strip())
    .merge(
        df_genotyping_str[["FID"]].assign(FID=lambda d: d["FID"].fillna("").str.strip()),
        left_on="SUBJECT_ID",
        right_on="FID",
        how="inner",
    )
)

print("nunique(SUBJECT_ID):", survey_ids[survey_ids != ""].nunique())
print("nunique(FID):", geno_ids[geno_ids != ""].nunique())
print("intersección de IDs:", len(ids_intersection))
print("filas del merge inner:", merge_inner.shape[0])

display(merge_inner[["SUBJECT_ID", "FID"]].head(5))

## Evidencia visible mínima (Apartado 1)
- Tabla con estado de los 2 archivos requeridos (encontrado/ruta).
- Tamaño de `survey` y `genotyping` (`shape`) cuando los archivos estén presentes.
- Lista de columnas comunes para justificar la relación entre tablas.
- Muestras `head(3)` de ambas tablas.
- Conteo de `-9` en genotipado como evidencia de missing codificado.

### Conclusiones (completar tras ejecutar)
- [Pendiente de ejecución] Confirmación de disponibilidad de ambos archivos.
- [Pendiente de ejecución] Clave de JOIN validada en datos reales.
- [Pendiente de ejecución] Estado inicial de calidad ligado al valor `-9`.